# `DDXRA`: Using `xarray` in `DiscreteDistribution`


First we import relevant libraries and tools, including the new `DiscreteDistributionXRA` class.


In [1]:
import numpy as np
from HARK.distribution import (
    MeanOneLogNormal,
    DiscreteDistributionXRA,
    calc_expectation,
    combine_indep_dstns,
)

We create a distribution of shocks to income from continuous distributions.


In [2]:
PermShkDstn = MeanOneLogNormal().approx(200)
TranShkDstn = MeanOneLogNormal().approx(200)
IncShkDstn = combine_indep_dstns(PermShkDstn, TranShkDstn)

Taking the components of `IncShkDstn`, we can now create a `DiscreteDistributionXRA` object. As a demonstration of additional features, we can add a name attribute to the `DDXRA` object, as well as named dimensions and coordinates.


In [3]:
x_dist = DiscreteDistributionXRA(
    IncShkDstn.pmf,
    IncShkDstn.X,
    name="Distribution of Shocks to Income",
    dims=("rv", "x"),
    coords={"rv": ["perm_shk", "tran_shk"]},
)

As a side note, we can also use set the boolean option `xarray = True` in `combine_indep_dstns` with the same attributes to create an `DDXRA` object in place.


In [4]:
x_dist = combine_indep_dstns(
    PermShkDstn,
    TranShkDstn,
    xarray=True,
    name="Distribution of Shocks to Income",
    dims=("rv", "x"),
    coords={"rv": ["perm_shk", "tran_shk"]},
)

The underlying object and metadata is stored in a `xarray.DataArray` object which can be accessed using the `.xarray` attribute.


In [5]:
x_dist.xarray

<xarray.DataArray 'Distribution of Shocks to Income' (rv: 2, x: 40000)>
array([[ 0.03491194,  0.03491194,  0.03491194, ..., 11.50651472,
        11.50651472, 11.50651472],
       [ 0.03491194,  0.0530801 ,  0.06439949, ...,  5.72401042,
         6.9659349 , 11.50651472]])
Coordinates:
  * rv       (rv) <U8 'perm_shk' 'tran_shk'
Dimensions without coordinates: x
Attributes:
    pmf:      [2.5e-05 2.5e-05 2.5e-05 ... 2.5e-05 2.5e-05 2.5e-05]
    seed:     0
    RNG:      RandomState(MT19937)

### Taking the Expected Value of `DDXRA` objects.


Taking the expectation of a `DDXRA` object is straightforward using the own `expected_value()` method.


In [6]:
x_dist.expected_value()

array([1., 1.])

As in the `DiscreteDistribution`, we can provide a function and arguments to the `expected_value()` method.


In [7]:
aGrid = np.linspace(0, 20, 100)
R = 1.03

In [8]:
%%timeit
x_dist.expected_value(lambda x, a, R: R * a / x[0] + x[1], aGrid, R)

15.8 ms ± 343 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


Compared to the old method of `calc_expectation` which takes a `DiscreteDistribution` object as input, the new method which takes a `DiscreteDistributionXRA` object remains significantly faster.

"""


In [9]:
%%timeit
calc_expectation(IncShkDstn, lambda x, a, R: R * a / x[0] + x[1], aGrid, R)

221 ms ± 3.15 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### Using functions with labels to take expresive expectations.


The main difference is that the `expected_value()` method of `DDXRA` objects can take a function that uses the labels of the `xarray.DataArray` object. This allows for clearer and more expresive mathematical functions and transition equations. Surprisingly, using a function with labels does not add much overhead to the function evaluation.


In [10]:
%%timeit
x_dist.expected_value(
    lambda x, a, R: R * a / x["perm_shk"] + x["tran_shk"], aGrid, R, labels=True
)

15.7 ms ± 159 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can also use `HARK.distribution.ExpectedValue`.

In [11]:
from HARK.distribution import ExpectedValue

In [12]:
ExpectedValue(
    lambda x, a, R: R * a / x["perm_shk"] + x["tran_shk"],
    dist=x_dist,
    args=(aGrid, R),
    labels=True,
)

array([ 1.        ,  1.56267794,  2.12535588,  2.68803382,  3.25071176,
        3.8133897 ,  4.37606764,  4.93874558,  5.50142352,  6.06410146,
        6.6267794 ,  7.18945734,  7.75213528,  8.31481322,  8.87749116,
        9.4401691 , 10.00284704, 10.56552498, 11.12820292, 11.69088086,
       12.2535588 , 12.81623674, 13.37891468, 13.94159262, 14.50427056,
       15.0669485 , 15.62962644, 16.19230438, 16.75498232, 17.31766026,
       17.8803382 , 18.44301614, 19.00569408, 19.56837202, 20.13104997,
       20.69372791, 21.25640585, 21.81908379, 22.38176173, 22.94443967,
       23.50711761, 24.06979555, 24.63247349, 25.19515143, 25.75782937,
       26.32050731, 26.88318525, 27.44586319, 28.00854113, 28.57121907,
       29.13389701, 29.69657495, 30.25925289, 30.82193083, 31.38460877,
       31.94728671, 32.50996465, 33.07264259, 33.63532053, 34.19799847,
       34.76067641, 35.32335435, 35.88603229, 36.44871023, 37.01138817,
       37.57406611, 38.13674405, 38.69942199, 39.26209993, 39.82